# Pandas Analysis and Reshaping 

## Overview 

Easier to work with a single `DataFrame` that you can slice, reshape, and aggregate.

No lists of `DataFrames`, `list`, etc.

<hr>

**`groupby`**:  Create aggreate tables with a single or multiple-index.

**`pivot`**: Create aggregate tables with two single or multiple-indecies.

**`melt`**: Reverse of `pivot`.

<hr>

`pivot` + *multiple index* is great for timeseries operations.


### Goal

Demonstrate how these concepts enabled the advanced analysis on a project with Vermont. 

Inspire this technique.

In [ ]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc={'figure.figsize':(12,6)})
plt.style.use(['seaborn-whitegrid'])

import warnings
warnings.simplefilter('ignore')

import utils

## Apply revisit

In [ ]:
filenames = ['data/measured_real_power.csv',
             'data/measured_reactive_power.csv']

df = utils.read_dir(filenames)
df.shape

In [ ]:
df.head()

### Pivoting and grouping

In [ ]:
pivot = df.pivot_table(index='timestamp', columns=['variable', 'attribute'], values='value')
pivot.head()

In [ ]:
def func(x):
    return x.name, x.shape

pivot.groupby(level=0, axis=1).apply(func)

In [ ]:
import math
def compute_mag(df):
    total =  (df['measured_real_power']*df['measured_real_power'] + 
              df['measured_reactive_power']*df['measured_reactive_power'])
    
    return total.apply(math.sqrt)

def func(x):
    return compute_mag(x[x.name])

mag = pivot.groupby(level=0, axis=1).apply(func)
mag.head()

In [ ]:
mag_stack = mag.reset_index().melt(id_vars='timestamp')
mag_stack['attribute'] = 'magnitude'
mag_stack.head()

In [ ]:
df.shape, mag_stack.shape

In [ ]:
pd.concat([df, mag_stack], axis=0).shape

## The Vermont Project 

### Characteristic

<hr>

Two feeders

Two seasons

1000 Waterheaters

Several control algorithms

Sometimes solar and batteries... 

**Many simulations**

### Questions

<hr>

What happends to the peak load?

What happends to the voltage?

What happends to the comfort of the customer?  We are **focusing** on this.

## The comfort data 

Original files on NREL's HPC

1 minute resolution for 14 days

One file per attribute (e.g. flow rate, water temperature)

This dataset is a collection of events where:

1. flow rate: **positive demand** (demand > 0) and **shower demand** (demand > 0.4)
2. temperature was below **110** and **105**

We use a `shift` value from the peak of a particular days to make the results align.

<br>


*Collected into a standard form for each simulation and concatenated into a single file*


<br>

The data

In [ ]:
df = pd.read_csv("data/comfort_data.csv.gz",  compression='gzip')

# limit the data a bit...
qry = ["solar==0",
       "days==14",
       "strategy in ['no_control', 'smooth', 'single_bucket']"]

df = df.query(" and ".join(qry))
df.timestamp = pd.to_datetime(df.timestamp)
df.shape

In [ ]:
df.head()

In [ ]:
(len(df.name.unique()), 
len(df.feeder.unique()), 
len(df.season.unique()),
len(df.strategy.unique()), 
len(df.control.unique()))

##  Reshaping with Pandas

`groupby`

`pivot_table`

`melt`

`applying`

Warm up with a single simulation... cuts down on the details.

In [ ]:
qry = ["feeder=='one'",
       "season=='summer'",
       "strategy=='no_control'",
       "control==0",
       "attribute=='shower demand 110'"]

tmp = df.query(" and ".join(qry))
tmp.shape

How many unique customers are impacted over the period?

In [ ]:
len(tmp.name.unique())

How long is the period?

In [ ]:
tmp.timestamp.max() - tmp.timestamp.min()

How many unique customer impacted per day?

#### `groupby`

In [ ]:
tmp.head()

In [ ]:
grp = tmp.groupby(['day', 'name'])['value'].count()
grp.head(20)

In [ ]:
grp.reset_index().groupby(['day'])['name'].count()

In [ ]:
grp.reset_index().groupby(['day'])['name'].count().plot(kind='barh')

In [ ]:
grp.reset_index().groupby(['day'])['name'].count().mean()

#### Another method  `pivot_table`

In [ ]:
piv = tmp.pivot_table(index='day', 
                      columns='name', 
                      values='value', 
                      aggfunc='count')
piv.head()

I count because I want unique customers not total events.

In [ ]:
piv.count(axis=1).plot(kind='barh')

A customer was impacted more than once.

In [ ]:
grp.reset_index().groupby(['day'])['name'].count().sum(), len(tmp.name.unique())

Events per customer?

In [ ]:
tmp.groupby(['name'])['value'].count().plot(kind='hist', bins=50)

Check on the number of events?

In [ ]:
tmp.groupby(['name'])['value'].count().sum()

## Question

### What is the average % of customers impacted as a function of time?

In [ ]:
grp = tmp.groupby(['shift', 'day', 'name'])['value'].count()
grp.head()

In [ ]:
grp2 = grp.reset_index().groupby(['shift', 'day'])['value'].sum()
grp2.head()

Make a pivot of the number of events per day (columns) as a function of shift (rows)

In [ ]:
piv = grp2.reset_index().pivot_table(index='shift', 
                                     columns='day', 
                                     values='value').fillna(0)
piv.head()

In [ ]:
piv.plot()

Check on the number of events.

In [ ]:
piv.sum().sum()

Events per minute.

In [ ]:
piv.fillna(0).mean(axis=1).plot()

####  What did we do?

In [ ]:
final = (tmp.groupby(['shift', 'day', 'name'])['value'].count()
            .reset_index().groupby(['shift', 'day'])['value'].sum()
            .reset_index().pivot_table(index='shift', 
                                       columns='day', 
                                       values='value').fillna(0)
            .mean(axis=1)
)

final.plot()

Using the multi-level pivot.

In [ ]:
piv = tmp.pivot_table(index='shift', 
                      columns=['day', 'name'],
                      values='value', 
                      aggfunc='count')
piv.head()

Check totals...

In [ ]:
piv.sum(axis=1).sum()

In [ ]:
piv1 = piv.sum(axis=1, level=0).fillna(0)
piv1.head()

In [ ]:
piv1.mean(axis=1).plot()

Recap in 4 steps

In [ ]:
final = (tmp.pivot_table(index='shift', 
                         columns=['day', 'name'],
                         values='value', 
                         aggfunc='count')
            .sum(axis=1, level=0)
            .fillna(0)
            .mean(axis=1))
        
final.plot()

**groupby + pivot**

```python
final = (tmp.groupby(['shift', 'day', 'name'])['value'].count()
            .reset_index().groupby(['shift', 'day'])['value'].sum()
            .reset_index().pivot_table(index='shift', 
                                       columns='day', 
                                       values='value')
            .fillna(0)
            .mean(axis=1)
)
```

**multi-level pivot**

```python
final = (tmp.pivot_table(index='shift', 
                         columns=['day', 'name'],
                         values='value', 
                         aggfunc='count')
            .sum(axis=1, level=0)
            .fillna(0)
            .mean(axis=1))
```

Number of events every 10 minutes.

In [ ]:
import datetime 

freq = "10min"

final = (tmp.pivot_table(index='shift', 
                         columns=['day', 'name'],
                         values='value', 
                         aggfunc='count')
            .sum(axis=1, level=0)
            .fillna(0))

dt_index = (pd.to_datetime("2013-07-01 13:00:00") + 
            final.index.map(lambda x: datetime.timedelta(hours=x)))

sample = final.set_index(dt_index).resample(freq).sum().fillna(0)
sample.index = (sample.index - pd.to_datetime("2013-07-01 13:00:00")).map(lambda x: x.total_seconds()/60./60.)

sample.mean(axis=1).plot()

What we did..

In [ ]:
final = tmp.pivot_table(index='shift', columns=['day', 'name'], values='value', aggfunc='count')
            
final = final.sum(axis=1, level=0).fillna(0)

dt_index = pd.to_datetime("2013-07-01 13:00:00") + final.index.map(lambda x: datetime.timedelta(hours=x))
sample = final.set_index(dt_index).resample(freq).sum().fillna(0)
sample.index = (sample.index - pd.to_datetime("2013-07-01 13:00:00")).map(lambda x: x.total_seconds()/60./60.)

sample.mean(axis=1).plot()

## Many simulations

In [ ]:
qry = ["feeder=='one'",
       "season=='summer'",
       "attribute=='shower demand 110'"]

tmp = df.query(" and ".join(qry))

In [ ]:
final = tmp.pivot_table(index='shift', 
                         columns=['feeder', 'season', 'strategy', 'control', 'day', 'name'], 
                         values='value', 
                         aggfunc='count')
final.head()

In [ ]:
final = final.sum(axis=1, level=[0,1,2,3,4]).fillna(0)
final.head()

In [ ]:
dt_index = pd.to_datetime("2013-07-01 13:00:00") + final.index.map(lambda x: datetime.timedelta(hours=x))
sample = final.set_index(dt_index).resample('15min').sum().fillna(0)
sample.index = (sample.index - pd.to_datetime("2013-07-01 13:00:00")).map(lambda x: x.total_seconds()/60./60.)
sample.head()

In [ ]:
sample = sample.mean(axis=1, level=[0,1,2,3])
sample.head()

In [ ]:
sample[('one', 'summer', 'no_control')].plot()

Look at all scenarios

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(10,6), sharex=True, sharey=True)

sample[('one', 'summer', 'no_control')].plot(ax=ax[0])
sample[('one', 'summer', 'smooth')].plot(ax=ax[1])
sample[('one', 'summer', 'single_bucket')].plot(ax=ax[2])

plt.tight_layout()

In [ ]:
sample['one'].plot()